In [1]:
import pandas as pd
import numpy as np
import datetime

#import function to query mysql server
from Functions.run_query import run_query
from Functions.update_sf import update_sf



#import SDR assignments and segments
query = "SELECT * FROM SALES.SDRAssignments"
data = run_query(query, database = 'SALES')

#pull all accounts for CX and EX AEs

ae_reps = data['RepId'][data['RepId'].isnull() == False]
ae_reps = set(ae_reps)
ae_reps = '", "'.join(ae_reps)
ae_reps = '"' + ae_reps + '"'

#CX
query = "SELECT Id, OwnerId, SDR_RS__c, SDR_RC_Rep__c, SDR_EI__c from SFDC.Account where OwnerId in (" + ae_reps + ")"
ae_accounts = run_query(query, database = 'SFDC')

#EX
query = "SELECT Id, X360_EE_Rep__c as 'OwnerId', SDR_RS__c, SDR_RC_Rep__c, SDR_EI__c from SFDC.Account where X360_EE_Rep__c in (" + ae_reps + ")"
ae_accounts_ex = run_query(query, database = 'SFDC') 
ae_accounts_ex.drop_duplicates().to_csv('SDRFiles/TrueUp/ae_accounts_ex' + '.csv', index=False)

#RC 
query = "SELECT Id, Panels_Rep__c as 'OwnerId', SDR_RS__c, SDR_RC_Rep__c, SDR_EI__c from SFDC.Account where Panels_Rep__c in (" + ae_reps + ")"
ae_accounts_rc = run_query(query, database = 'SFDC')

#Join groups into single frame
frames = [ae_accounts, ae_accounts_ex, ae_accounts_rc]
ae_accounts = pd.concat(frames)
ae_accounts.drop_duplicates(inplace = True)
ae_accounts.reset_index(drop=True, inplace=True)


#merge correct SDR owner onto accounts
ae_accounts = ae_accounts.merge(data[['RepId', 'CX_SDR']][data['CX_SDR'].isnull() == False], how='left', left_on='OwnerId', right_on='RepId')
ae_accounts = ae_accounts.merge(data[['RepId', 'RC_SDR']][data['RC_SDR'].isnull() == False], how='left', left_on='OwnerId', right_on='RepId')
ae_accounts = ae_accounts.merge(data[['RepId', 'EX_SDR']][data['EX_SDR'].isnull() == False], how='left', left_on='OwnerId', right_on='RepId')
ae_accounts = ae_accounts.merge(data[['RepId', 'ENT_SDR']][data['ENT_SDR'].isnull() == False], how='left', left_on='OwnerId', right_on='RepId')


#Break out accounts dataframe into separate products for individual upload
cx_true_up = ae_accounts[['Id', 'CX_SDR']][(ae_accounts['SDR_RS__c'] != ae_accounts['CX_SDR']) & (ae_accounts['CX_SDR'].isnull() == False)]
cx_true_up.columns = ['Id', 'SDR_RS__c']
cx_true_up.reset_index(inplace = True, drop = True)
ex_true_up = ae_accounts[['Id', 'EX_SDR']][(ae_accounts['SDR_EI__c'] != ae_accounts['EX_SDR']) & (ae_accounts['EX_SDR'].isnull() == False)]
ex_true_up.columns = ['Id', 'SDR_EI__c']
rc_true_up = ae_accounts[['Id', 'RC_SDR']][(ae_accounts['SDR_RC_Rep__c'] != ae_accounts['RC_SDR']) & (ae_accounts['RC_SDR'].isnull() == False)]
rc_true_up.columns = ['Id', 'SDR_RC_Rep__c'] 
ent_true_up = ae_accounts[['Id', 'ENT_SDR']][((ae_accounts['SDR_RS__c'] != ae_accounts['ENT_SDR']) | (ae_accounts['SDR_RC_Rep__c'] != ae_accounts['ENT_SDR']) | (ae_accounts['SDR_EI__c'] != ae_accounts['ENT_SDR'])) & (ae_accounts['ENT_SDR'].isnull() == False)]
ent_true_up['ENT_SDR_2'] = ent_true_up['ENT_SDR']
ent_true_up['ENT_SDR_3'] = ent_true_up['ENT_SDR']
ent_true_up.columns = ['Id', 'SDR_RS__c', 'SDR_EI__c','SDR_RC_Rep__c']


#Create upload files for each group with current date
now = datetime.datetime.now()
cx_true_up.drop_duplicates().to_csv('SDRFiles/TrueUp/cx_sdr' + str(now.month) + '_' + str(now.day) + '_' + str(now.year) + '.csv', index=False)
ex_true_up.drop_duplicates().to_csv('SDRFiles/TrueUp/ex_sdr' + str(now.month) + '_' + str(now.day) + '_' + str(now.year) + '.csv', index=False)
rc_true_up.drop_duplicates().to_csv('SDRFiles/TrueUp/rc_sdr' + str(now.month) + '_' + str(now.day) + '_' + str(now.year) + '.csv', index=False)
ent_true_up.drop_duplicates().to_csv('SDRFiles/TrueUp/ent_sdr' + str(now.month) + '_' + str(now.day) + '_' + str(now.year) + '.csv', index=False)
ae_accounts.drop_duplicates().to_csv('SDRFiles/TrueUp/ae_accounts' + str(now.month) + '_' + str(now.day) + '_' + str(now.year) + '.csv', index=False)


#Print out the number of accounts being updated for each group
print("CX: " + str(len(cx_true_up['Id'].drop_duplicates())) + " accounts")
print("RC: " + str(len(rc_true_up['Id'].drop_duplicates())) + " accounts")
print("EX: " + str(len(ex_true_up['Id'].drop_duplicates())) + " accounts")
print("ENT: " + str(len(ent_true_up['Id'].drop_duplicates())) + " accounts")


from IPython.display import Audio
sound_file = "LOZ_Secret.wav"
Audio(filename = sound_file, rate = 1, autoplay = True)

/Users/jnichols/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/jnichols/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/jnichols/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/jnichols/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy 

CX: 46765 accounts
RC: 77620 accounts
EX: 49876 accounts
ENT: 41507 accounts


In [9]:
from IPython.display import Audio
sound_file = "LOZ_Secret.wav"
Audio(filename = sound_file, rate = 1, autoplay = True)